In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import joblib


In [2]:
def connection():
        """This methos creates a connection with the  db with pandas"""
        host = 'localhost'
        port=3308
        user = 'house'
        password = 'house'
        db = 'house'
        try:
            connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{db}'
            engine = create_engine(connection_string)
        
        except Exception as e:
            print((f"Error creating connection: {e}"))
            raise

        return engine

In [3]:
engine = connection() 

       

In [26]:
df = pd.read_sql_table('house_WareHouse', con=engine)
        

In [27]:
len(df.columns)

71

In [ ]:


def robustScaler(dataframe):
    """This method scales the data using RobustScaler and helps to reduce the influence of outliers"""
    Robustscaler = RobustScaler()
    Numeric_scaled = Robustscaler.fit_transform(dataframe)
    dataScaled = pd.DataFrame(Numeric_scaled,columns=dataframe.columns)

    return dataScaled

In [8]:
Y = df[['SalePrice']]

In [9]:
X = df.drop(columns=['SalePrice','Id'])

In [10]:
X.isnull().sum()

MSSubClass        0
LotFrontage     259
LotArea           0
OverallQual       0
OverallCond       0
               ... 
HeatingQC         0
KitchenQual       0
GarageFinish      0
GarageQual        0
GarageCond        0
Length: 69, dtype: int64

In [11]:
X = X.fillna(0)

In [12]:
X= robustScaler(X)

In [15]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=42)

In [13]:
# Generate a instance of the model RandomForestRegressor
model = RandomForestRegressor(n_estimators=200,max_depth=15, random_state=42,max_features='sqrt',n_jobs=-1,oob_score=True)


In [16]:
# Train the model
model.fit(X_train,Y_train.values.ravel())

RandomForestRegressor(max_depth=15, max_features='sqrt', n_estimators=200,
                      n_jobs=-1, oob_score=True, random_state=42)

In [17]:
# Predict the test set results
Y_test_pred = model.predict(X_test)

In [18]:
# Evaluate the model
mse = mean_squared_error(Y_test, Y_test_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, Y_test_pred)
r2 = r2_score(Y_test, Y_test_pred)

# Mostrar métricas
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R^2: {r2:.2f}")

MSE: 444367268.31
RMSE: 21080.02
MAE: 10730.38
R^2: 0.94


In [38]:
Y.mean()

SalePrice    180921.19589
dtype: float64

In [19]:
# Save the model
joblib.dump(model, 'model.pkl')

['model.pkl']

In [6]:
train = pd.read_csv(r"C:\Users\rogel\Documents\GitHub\House-Prices-Prediction\train.csv")

In [7]:
len(train.columns)

81

In [8]:
test = pd.read_csv(r"C:\Users\rogel\Documents\GitHub\House-Prices-Prediction\test.csv")

In [13]:
len(test.columns)

80

In [30]:
columns_to_drop = test.columns.difference(df.columns)

In [31]:
columns_to_drop

Index(['CentralAir', 'LandContour', 'LotConfig', 'MasVnrType', 'PavedDrive',
       'Street'],
      dtype='object')

In [15]:
    #Get the columns that have more than 80% of ZEROS
def columnas_ceros(dataframe):

        train_columns = dataframe.columns

        columnas = [col for col in train_columns if (dataframe[col]==0).sum() > 650 ]

        return columnas
    
    #Get the columns that have more than 80% of NULLS
def columnas_nulos(dataframe):

        train_columns = dataframe.columns

        columnas = [col for col in train_columns if dataframe[col].isna().sum() > 650 ]

        return columnas

In [16]:
columnas_ceros = columnas_ceros(train)
columnas_nulos = columnas_nulos(train)

In [24]:
columnas_ceros.extend(columnas_nulos)

In [25]:
columnas_ceros

['MasVnrArea',
 'BsmtFinSF2',
 '2ndFlrSF',
 'LowQualFinSF',
 'BsmtFullBath',
 'BsmtHalfBath',
 'HalfBath',
 'Fireplaces',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'Alley',
 'FireplaceQu',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [22]:
test.drop(columns=columnas, inplace=True)

In [23]:
len(test.columns)

60